In [12]:
import pandas as pd
import tiledb
tiledb.libtiledb.version()

(1, 4, 0)

In [13]:
!aws s3 ls --human-readable --summarize s3://tiledb-presto/nyse/tiledb-arrays/quote_20180730/

                           PRE __1a0afb84dfc84ae89b6630dbff7f55e1_1539301371490/
2018-10-12 12:08:43  522 Bytes __array_schema.tdb
2018-10-12 12:08:43    0 Bytes __lock.tdb

Total Objects: 2
   Total Size: 522 Bytes


In [14]:
!aws s3 ls --human-readable --summarize "s3://tiledb-presto/nyse/tiledb-arrays/quote_20180730/__1a0afb84dfc84ae89b6630dbff7f55e1_1539301371490/"


2018-10-12 12:07:48    1.5 GiB Bid_Price.tdb
2018-10-12 12:07:48  630.3 MiB Bid_Size.tdb
2018-10-12 12:07:48  336.6 MiB Exchange.tdb
2018-10-12 12:07:48  516.6 KiB FINRA_ADF_MPID_Indicator.tdb
2018-10-12 12:07:48  117.7 MiB FINRA_ADF_Market_Participant_Quote_Indicator.tdb
2018-10-12 12:07:48    4.0 MiB FINRA_ADF_Timestamp.tdb
2018-10-12 12:07:48  156.2 MiB FINRA_BBO_Indicator.tdb
2018-10-12 12:07:51  117.7 MiB LULD_BBO_Indicator.tdb
2018-10-12 12:07:53  203.4 MiB National_BBO_Ind.tdb
2018-10-12 12:07:55   33.7 MiB National_BBO_LULD_Indicator.tdb
2018-10-12 12:07:57    1.5 GiB Offer_Price.tdb
2018-10-12 12:07:58  491.4 MiB Offer_Size.tdb
2018-10-12 12:08:00    2.0 GiB Participant_Timestamp.tdb
2018-10-12 12:08:06  117.7 MiB Quote_Cancel_Correction.tdb
2018-10-12 12:08:10    2.6 MiB Quote_Condition.tdb
2018-10-12 12:08:10  183.3 MiB Retail_Interest_Indicator.tdb
2018-10-12 12:08:13  117.9 MiB SIP_Generated_Message_Identifier.tdb
2018-10-12 12:08:15  516.9 KiB Security_Status_Indicator.td

In [15]:
ctx = tiledb.Ctx()
schema = tiledb.ArraySchema.load(ctx, "s3://tiledb-presto/nyse/tiledb-arrays/quote_20180730")

In [16]:
domain = schema.domain
pd.DataFrame([(domain.dim(i).name, domain.dim(i).dtype) for i in range(domain.ndim)], columns=("Dimension", "dtype"))

,Dimension,dtype
0,date,uint64
1,Time,uint64
2,Sequence_Number,uint64


In [17]:
attr_name_dtype = [(schema.attr(i).name, schema.attr(i).dtype) for i in range(schema.nattr)]
pd.DataFrame(attr_name_dtype, columns=("Attribute", "dtype"))

,Attribute,dtype
0,Exchange,|S0
1,Symbol,|S0
2,Bid_Price,float64
3,Bid_Size,uint32
4,Offer_Price,float64
5,Offer_Size,uint64
6,Quote_Condition,|S0
7,National_BBO_Ind,|S0
8,FINRA_BBO_Indicator,|S0
9,FINRA_ADF_MPID_Indicator,uint8


In [18]:
import prestodb

class PrestoConnection(object):
    
    def __init__(self, hostname=None, port=None, user=None):
        self.hostname = hostname
        self.port = port
        if user is None:
            user = 'presto'
        self.user = 'presto'
        self.catalog = 'tiledb'
        self.schema = 'tiledb'
        
    def query(self, sql_query):
       """Executes the query and returns a Pandas DataFrame of the results."""
       try:
           conn = prestodb.dbapi.connect(host=self.hostname,
                                         port=self.port,
                                         user=self.user,
                                         catalog=self.catalog,
                                         schema=self.schema,
                                         isolation_level=1)
           cur = conn.cursor()
           cur.execute(sql_query)
           query_results = cur.fetchall()
           colnames = [part[0] for part in cur.description]
           df = pd.DataFrame(query_results, columns=colnames)
           return df
       finally:
           cur.cancel()
           conn.close()

In [19]:
conn = PrestoConnection(hostname='172.31.65.206', port=8889)
conn.query('SELECT * FROM system.runtime.nodes')

,node_id,http_uri,node_version,coordinator,state
0,i-097e6c98deb321ad8,http://172.31.76.48:8889,0.206,False,active
1,i-0098cd5d8e669665d,http://172.31.78.41:8889,0.206,False,active
2,i-0c3ffd668a20836ab,http://172.31.65.206:8889,0.206,True,active


In [20]:
conn.query("""
    DESCRIBE "s3://tiledb-presto/nyse/tiledb-arrays/quote_20180730"
""")

,Column,Type,Extra,Comment
0,date,bigint,,Dimension
1,time,bigint,,Dimension
2,sequence_number,bigint,,Dimension
3,exchange,varchar(1),,Attribute
4,symbol,varchar,,Attribute
5,bid_price,double,,Attribute
6,bid_size,bigint,,Attribute
7,offer_price,double,,Attribute
8,offer_size,bigint,,Attribute
9,quote_condition,varchar(1),,Attribute


In [21]:
with tiledb.SparseArray(ctx, "s3://tiledb-presto/nyse/tiledb-arrays/quote_20180730", mode='r') as array:
    res = array.query(
        attrs=("Bid_Price", "Bid_Size", "Offer_Price", "Offer_Size"))[:, 94500000000000:94600000000000 , :]
pd.DataFrame(res)[0:10]

,coords,Bid_Price,Bid_Size,Offer_Price,Offer_Size
0,"(20180730, 94500000026000, 179489301)",23.42,4,23.57,1
1,"(20180730, 94500000068315, 1723529)",61.03,1,61.08,1
2,"(20180730, 94500000085360, 2506429)",15.05,1,15.15,1
3,"(20180730, 94500000100768, 2273321)",60.35,1,60.95,2
4,"(20180730, 94500000189529, 2273322)",60.65,1,60.95,2
5,"(20180730, 94500000209225, 2273323)",26.52,1,26.60,1
6,"(20180730, 94500000244105, 1723530)",7.81,1,7.83,1
7,"(20180730, 94500000252412, 2273324)",26.52,2,26.60,1
8,"(20180730, 94500000273554, 2273325)",37.40,1,37.80,1
9,"(20180730, 94500000304003, 1723531)",7.81,1,7.82,4


In [22]:
conn.query("""
    SELECT Date, Time, Sequence_Number, 
           Bid_Price, Bid_Size, Offer_Price, Offer_Size
    FROM "s3://tiledb-presto/nyse/tiledb-arrays/quote_20180730" 
    WHERE Time between 94500000000000 AND 94600000000000 limit 10
""")

,Date,Time,Sequence_Number,Bid_Price,Bid_Size,Offer_Price,Offer_Size
0,20180730,94500000026000,179489301,23.42,4,23.57,1
1,20180730,94500000068315,1723529,61.03,1,61.08,1
2,20180730,94500000085360,2506429,15.05,1,15.15,1
3,20180730,94500000100768,2273321,60.35,1,60.95,2
4,20180730,94500000189529,2273322,60.65,1,60.95,2
5,20180730,94500000209225,2273323,26.52,1,26.60,1
6,20180730,94500000244105,1723530,7.81,1,7.83,1
7,20180730,94500000252412,2273324,26.52,2,26.60,1
8,20180730,94500000273554,2273325,37.40,1,37.80,1
9,20180730,94500000304003,1723531,7.81,1,7.82,4
